# How to start

Before starting you must ensure that `scipp` and `mantid` are on your `PYTHONPATH`.

For `scipp` and `mantid` follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.

Additionally, the other package dependencies are `fabio` and `astropy`: `conda install fabio astropy`

This notebook follows the steps and uses the parameters from this reduction script https://git.esss.dk/testbeamline/gp2/blob/master/reduce.py but have them converted using equivalent `scipp` functionality.

Before starting you need to configure:

- The experiment directory in the `experiment_dir` variable - this 

In [1]:
import os

# Specify where the data can be found
experiment_dir = "/home/dtasev/dev/scipp_imaging/RAL_Mantid_Sep2018/"

# let's get the process started:
tofs_path = os.path.join(experiment_dir, 'data_GP2', 'metadata', 'GP2_BCC_time_values.txt')
sample_path = os.path.join(experiment_dir, 'data_GP2', 'Timeslices WFM BBC Steel')
ob_path = os.path.join(experiment_dir, 'data_GP2', 'Timeslices WFM Open Beam')

# defining grouping of 2D detector pixels
grouping_number = 3
nx_target = grouping_number
ny_target = grouping_number

frame_parameters = [(15167,23563,64),
                    (24393,32758,64),
                    (33365,40708,64),
                    (41410,48019,64),
                    (49041,55311,64),
                    (56077,59872,64)]

# Two options availabe for frame shift increments: 1) Robins Calibrated Values 2) Owens Calculated Values.

frame_shift_increments = [-6630.0, -2420.0, -2253.0, -2095.0, -1946.0, -1810.0]
# frame_shift_increments = [-6630,-2423,-2252,-2058,-1949,-1576]
frame_shifts = [sum(frame_shift_increments[:i + 1]) for i in range(len(frame_shift_increments))]

# rebin region reduced to cut off frames that contain no data
rebin_parameters = (8500, 43000, 64)

In [2]:
# created by Owen Arnold (optional e-mail) on 2018-09-19
# last modified by Peter M. Kadletz (peter.kadletz@esss.se) on 2018-11-12
import fabio
import numpy as np
import glob
import sys
import csv

import scipp as sc
import numpy as np
from scipp import Dim

from mantid.simpleapi import *
from mantid.api import AlgorithmManager

In [3]:
# Some helper functions

def read_x_values(tof_file):
    """
    Reads the TOF values from the CSV into a list
    """
    tof_values = list()
    with open(tof_file) as fh:
        csv_reader = csv.reader(fh, delimiter='\t')
        next(csv_reader, None)  # skip header
        for row in csv_reader:
            tof_values.append(float(row[1]))
    return tof_values


def _load_tiffs(tiff_dir):
    if not os.path.isdir(tiff_dir):
        raise RuntimeError(tiff_dir + " is not directory")
    stack = []
    path_length = len(tiff_dir) + 1
    filenames = sorted(glob.glob(tiff_dir + "/*.tiff"))
    nfiles = len(filenames)
    count = 0
    print(f"Loading {nfiles} files from '{tiff_dir}'")
    for filename in filenames:
        count += 1
        print('\r{0}: Image {1}, of {2}'.format(filename[path_length:], count, nfiles), end="")
        img = fabio.open(os.path.join(tiff_dir, filename))
        stack.append(np.flipud(img.data))

    return np.array(stack)

def tiffs_to_variable(tiff_dir):
    """
    Loads all tiff images from the directory into a scipp Variable.
    """
    stack = _load_tiffs(tiff_dir)
    return sc.Variable([Dim.Tof, Dim.Spectrum], 
                       values=stack.astype(np.float64).reshape(
                           stack.shape[0], stack.shape[1]*stack.shape[2]))

from functools import reduce

def stitch(data_array, frame_parameters, rebin_parameters):
    """
    Stitches the 5 different frames data.
    
    It crops out each frame, then shifts it so that all frames align,
    and then rebins to the common bins used for all frames.
    """
    frames = []

    rebin_params = sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))
    
    for i, (slice_bins, shift_parameter) in enumerate(zip(frame_parameters, frame_shifts)):
        bins = sc.Variable([Dim.Tof], values=np.arange(*slice_bins, dtype=np.float64))
        # Rebins the whole data to crop it to frame bins
        rebinned = sc.rebin(data_array, Dim.Tof, bins)
        # Shift the frame backwards to make all frames overlap
        rebinned.coords[Dim.Tof] += shift_parameter
        # Rebin to overarching coordinates so that the frame coordinates align
        rebinned = sc.rebin(rebinned, Dim.Tof, rebin_params)

        frames.append(rebinned)

    for f in frames[1:]:
        frames[0] += f

    return frames[0]

def remove_special_values(values):
    np.nan_to_num(values, copy=False)
    values[values == sys.float_info.max] = 0

In [4]:
ds = sc.Dataset()

# Load X values from the TOF file
ds.coords[Dim.Tof] = sc.Variable([Dim.Tof], unit=sc.units.us, values=read_x_values(tofs_path))
ds.coords[Dim.Tof] *= 1e3

In [5]:
ds["reference"] = tiffs_to_variable(ob_path)
ds["sample"] = tiffs_to_variable(sample_path)

Loading 2001 files from '/home/dtasev/dev/scipp_imaging/RAL_Mantid_Sep2018/data_GP2/Timeslices WFM Open Beam'


GP2_007-003-WFM-open-beam_0488.tiff: Image 489, of 2001

GP2_007-003-WFM-open-beam_0737.tiff: Image 738, of 2001

GP2_007-003-WFM-open-beam_0990.tiff: Image 991, of 2001

GP2_007-003-WFM-open-beam_1249.tiff: Image 1250, of 2001

...

GP2_007-004-WFM-open-beam_1029.tiff: Image 1030, of 2001

GP2_007-004-WFM-open-beam_1288.tiff: Image 1289, of 2001

GP2_007-004-WFM-open-beam_1547.tiff: Image 1548, of 2001

GP2_007-004-WFM-open-beam_1805.tiff: Image 1806, of 2001

GP2_007-004-WFM-open-beam_2000.tiff: Image 2001, of 2001

In [6]:
ds.coords[Dim.Spectrum] = sc.Variable([Dim.Spectrum], values=np.arange(ds["sample"].shape[1]))
ds

<scipp.Dataset>
Dimensions: {{Dim.Spectrum, 104976}, {Dim.Tof, 2001}}
Coordinates:
    Dim.Tof                   float64    [μs]            (Dim.Tof [bin-edges])  [15.000000, 44.990000, ..., 59985.010000, 60015.000000]
    Dim.Spectrum              int64      [dimensionless]  (Dim.Spectrum)  [0, 1, ..., 104974, 104975]
Data:
    reference                 float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    sample                    float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]

In [7]:
stitched = sc.Dataset(coords={Dim.Tof: sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))})

stitched["sample"] = stitch(ds["sample"], frame_parameters, rebin_parameters)
stitched["reference"] = stitch(ds["reference"], frame_parameters, rebin_parameters)

stitched

<scipp.Dataset>
Dimensions: {{Dim.Spectrum, 104976}, {Dim.Tof, 539}}
Coordinates:
    Dim.Tof                   float64    [dimensionless]  (Dim.Tof [bin-edges])  [8500.000000, 8564.000000, ..., 42932.000000, 42996.000000]
    Dim.Spectrum              int64      [dimensionless]  (Dim.Spectrum)  [0, 1, ..., 104974, 104975]
Data:
    reference                 float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    sample                    float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]

In [8]:
stitched["normalized"] = stitched["sample"] / stitched["reference"]

remove_special_values(stitched["normalized"].values)
np.max(stitched["normalized"].values)

10388.364744067625

In [9]:
stitched["normalized_summed"] = sc.sum(stitched["normalized"], Dim.Spectrum)

In [10]:
def make_detector_groups(nx_original, ny_original, nx_target, ny_target):
    element_width_x = nx_original // nx_target
    element_width_y = ny_original // ny_target
    
    # To contain our new spectra mappings
    grid = np.zeros((nx_original, ny_original), dtype=np.float64)

    for i in range(0, nx_target):
        for j in range(0, ny_target):
            x_start = i * element_width_x
            x_end = (i + 1) * element_width_x
            
            y_start = j * element_width_y
            y_end = (j + 1) * element_width_y

            vals = np.full((element_width_x, element_width_y), i + j * nx_target, dtype=np.float64)
            grid[x_start:x_end, y_start:y_end] = vals
    
    return sc.Variable([Dim.Spectrum], values=grid.flatten())


stitched.labels["detector_mapping"] = make_detector_groups(324, 324, nx_target, ny_target)
stitched.labels["detector_mapping"]

<scipp.VariableProxy>     float64    [dimensionless]  (Dim.Spectrum)  [0.000000, 0.000000, ..., 8.000000, 8.000000]

In [11]:
stitched

<scipp.Dataset>
Dimensions: {{Dim.Spectrum, 104976}, {Dim.Tof, 539}}
Coordinates:
    Dim.Tof                   float64    [dimensionless]  (Dim.Tof [bin-edges])  [8500.000000, 8564.000000, ..., 42932.000000, 42996.000000]
    Dim.Spectrum              int64      [dimensionless]  (Dim.Spectrum)  [0, 1, ..., 104974, 104975]
Labels:
    detector_mapping          float64    [dimensionless]  (Dim.Spectrum)  [0.000000, 0.000000, ..., 8.000000, 8.000000]
Data:
    normalized                float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    normalized_summed         float64    [dimensionless]  (Dim.Tof)  [380.929957, 2005.506843, ..., 0.000000, 0.000000]
    reference                 float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    sample                    float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]

In [12]:
# Dim.Row is the target Dim that the output will be using
# There isn't a more relevant Dim, but maybe a DetectorGroup could
# be added?
dm1d = sc.groupby(stitched["normalized"], "detector_mapping", Dim.Row)
stitched["normalized_grpd"] = dm1d.sum(Dim.Spectrum)

In [13]:
def get_pos(pos):
    return [pos.X(), pos.Y(), pos.Z()]

def make_component_info(ws):
    sourcePos = ws.componentInfo().sourcePosition()
    samplePos = ws.componentInfo().samplePosition()

    def as_var(pos):
        return sc.Variable(value=np.array(get_pos(pos)),
                           dtype=sc.dtype.vector_3_float64,
                           unit=sc.units.m)

    return as_var(sourcePos), as_var(samplePos)

def init_pos(ws):
    nHist = ws.getNumberHistograms()
    pos = np.zeros([nHist, 3])

    spec_info = ws.spectrumInfo()
    for i in range(nHist):
        if spec_info.hasDetectors(i):
            p = spec_info.position(i)
            pos[i, :] = [p.X(), p.Y(), p.Z()]
        else:
            pos[i, :] = [np.nan, np.nan, np.nan]
    return sc.Variable([sc.Dim.Spectrum],
                       values=pos,
                       unit=sc.units.m,
                       dtype=sc.dtype.vector_3_float64)

def load_component_info_from_instrument_file(ds, file):
    try:
        import mantid.simpleapi as mantid
        from mantid.api import Workspace
    except ImportError:
        raise ImportError(
            "Mantid Python API was not found, please install Mantid framework "
            "as detailed in the installation instructions (https://scipp."
            "readthedocs.io/en/latest/getting-started/installation.html)")
    ws = mantid.Load(file)
    
    source_pos, sample_pos = make_component_info(ws)
    
    ds.labels["source_position"] = source_pos
    ds.labels["sample_position"] = sample_pos
    ds.labels["position"] = init_pos(ws)


In [14]:
load_component_info_from_instrument_file(stitched, '/home/dtasev/dev/scipp_imaging/RAL_Mantid_Sep2018/IDF/V20_Definition_GP2.xml')
stitched

<scipp.Dataset>
Dimensions: {{Dim.Spectrum, 104976}, {Dim.Row, 9}, {Dim.Tof, 539}}
Coordinates:
    Dim.Spectrum              int64      [dimensionless]  (Dim.Spectrum)  [0, 1, ..., 104974, 104975]
    Dim.Row                   float64    [dimensionless]  (Dim.Row)  [0.000000, 1.000000, ..., 7.000000, 8.000000]
    Dim.Tof                   float64    [dimensionless]  (Dim.Tof [bin-edges])  [8500.000000, 8564.000000, ..., 42932.000000, 42996.000000]
Labels:
    source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
    detector_mapping          float64    [dimensionless]  (Dim.Spectrum)  [0.000000, 0.000000, ..., 8.000000, 8.000000]
    position                  vector_3_float64  [m]              (Dim.Spectrum)  [(-0.011305, -0.011305, 0.600500), (-0.011235, -0.011305, 0.600500), ..., (0.011235, 0.011305, 0.600500), (0.011305, 0.011305, 0.600500)]
    sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]
Data:
    normalized                float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    normalized_grpd           float64    [dimensionless]  (Dim.Tof, Dim.Row)  [65.458559, 71.634779, ..., 0.000000, 0.000000]
    normalized_summed         float64    [dimensionless]  (Dim.Tof)  [380.929957, 2005.506843, ..., 0.000000, 0.000000]
    reference                 float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    sample                    float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]

In [15]:
# makes the position a DataConstProxy otherwise groupby won't take it
stitched["position"] = stitched.labels["position"]
dm1d = sc.groupby(stitched["position"], "detector_mapping", Dim.Row)
position = dm1d.mean(Dim.Spectrum)

# can't do stitched.labels["position"] = position because Labels won't take a DataArray
# also can't do stitched.labels["position"] = sc.Variable(position) because then sc.convert 
# thinks stitched.labels["position"] is dimensionless (as it's actually still a DataArray)

pos = np.zeros((position.shape[0], 3))
for i, val in enumerate(position.values):
    pos[i, :] = val

# finally add it back!
stitched.labels["position"] = sc.Variable(position.dims, pos, unit=sc.units.m, dtype=position.dtype)

In [16]:
%time stitched["normalized_wl"] = sc.neutron.convert(stitched["normalized_grpd"], Dim.Tof, Dim.Wavelength)
stitched

CPU times: user 0 ns, sys: 223 µs, total: 223 µs
Wall time: 229 µs


<scipp.Dataset>
Dimensions: {{Dim.Row, 9}, {Dim.Wavelength, 539}, {Dim.Tof, 539}, {Dim.Spectrum, 104976}}
Coordinates:
    Dim.Spectrum              int64      [dimensionless]  (Dim.Spectrum)  [0, 1, ..., 104974, 104975]
    Dim.Wavelength            float64    [Å μs^-1]     (Dim.Row, Dim.Wavelength [bin-edges])  [1.765094, 1.778385, ..., 8.915180, 8.928471]
    Dim.Tof                   float64    [dimensionless]  (Dim.Tof [bin-edges])  [8500.000000, 8564.000000, ..., 42932.000000, 42996.000000]
    Dim.Row                   float64    [dimensionless]  (Dim.Row)  [0.000000, 1.000000, ..., 7.000000, 8.000000]
Labels:
    source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
    detector_mapping          float64    [dimensionless]  (Dim.Spectrum)  [0.000000, 0.000000, ..., 8.000000, 8.000000]
    position                  vector_3_float64  [m]              (Dim.Row)  [(-0.007560, -0.007560, 0.600500), (-0.007560, 0.000000, 0.600500), ..., (0.007560, 0.000000, 0.600500), (0.007560, 0.007560, 0.600500)]
    sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]
Data:
    normalized                float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    normalized_grpd           float64    [dimensionless]  (Dim.Tof, Dim.Row)  [65.458559, 71.634779, ..., 0.000000, 0.000000]
    normalized_summed         float64    [dimensionless]  (Dim.Tof)  [380.929957, 2005.506843, ..., 0.000000, 0.000000]
    normalized_wl             float64    [dimensionless]  (Dim.Wavelength, Dim.Row)  [65.458559, 71.634779, ..., 0.000000, 0.000000]
    Attributes:
            source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
            sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]
            position                  vector_3_float64  [m]              (Dim.Row)  [(-0.007560, -0.007560, 0.600500), (-0.007560, 0.000000, 0.600500), ..., (0.007560, 0.000000, 0.600500), (0.007560, 0.007560, 0.600500)]
    position                  vector_3_float64  [m]              (Dim.Spectrum)  [(-0.011305, -0.011305, 0.600500), (-0.011235, -0.011305, 0.600500), ..., (0.011235, 0.011305, 0.600500), (0.011305, 0.011305, 0.600500)]
    reference                 float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    sample                    float64    [dimensionless]  (Dim.Tof, Dim.Spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]

In [18]:
stitched["normalized_wl"]

<scipp.DataProxy>
Dimensions: {{Dim.Wavelength, 539}, {Dim.Row, 9}}
Coordinates:
    Dim.Row                   float64    [dimensionless]  (Dim.Row)  [0.000000, 1.000000, ..., 7.000000, 8.000000]
    Dim.Wavelength            float64    [Å μs^-1]     (Dim.Row, Dim.Wavelength [bin-edges])  [1.765094, 1.778385, ..., 8.915180, 8.928471]
Labels:
    source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
    position                  vector_3_float64  [m]              (Dim.Row)  [(-0.007560, -0.007560, 0.600500), (-0.007560, 0.000000, 0.600500), ..., (0.007560, 0.000000, 0.600500), (0.007560, 0.007560, 0.600500)]
    sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]
Attributes:
    source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
    sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]
    position                  vector_3_float64  [m]              (Dim.Row)  [(-0.007560, -0.007560, 0.600500), (-0.007560, 0.000000, 0.600500), ..., (0.007560, 0.000000, 0.600500), (0.007560, 0.007560, 0.600500)]
Data:
    normalized_wl             float64    [dimensionless]  (Dim.Wavelength, Dim.Row)  [65.458559, 71.634779, ..., 0.000000, 0.000000]
    Attributes:
            source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
            sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]
            position                  vector_3_float64  [m]              (Dim.Row)  [(-0.007560, -0.007560, 0.600500), (-0.007560, 0.000000, 0.600500), ..., (0.007560, 0.000000, 0.600500), (0.007560, 0.007560, 0.600500)]

In [23]:
workspace_name = "normalized_wl"
x_coords = stitched["normalized_wl"].coords[stitched["normalized_wl"].dims[0]]
x_dim = stitched["normalized_wl"].dims[0]
x = x_coords.values

y = np.transpose(stitched["normalized_wl"].values)

ws1 = sc.compat.mantid.to_matrix_workspace(x, y, None, x_dim, workspace_name)

CompareWorkspaces_returns(Result=True, Messages=TableWorkspace
Columns: 3
Rows: 0
0 kB)

In [20]:
SaveNexus(ws, "scipp_normalized_wl.nxs")

In [21]:
fit_ws_name = 'fitted'
Fit(Function='name=LinearBackground,A0=5000,A1=0;name=UserFunction,Formula=h*erfc(a*(x-x0)),h=5000,a=-0.5,x0=4',
InputWorkspace=ws, Output=fit_ws_name, WorkspaceIndex=4, StartX=3.6, EndX=4.4)

Fit_returns(OutputStatus='Changes in function value are too small', OutputChi2overDoF=111.13443645451342, OutputNormalisedCovarianceMatrix=TableWorkspace
Columns: 6
Rows: 5
0 kB, OutputParameters=TableWorkspace
Columns: 3
Rows: 6
0 kB, OutputWorkspace=Workspace2D
Title: 
Histograms: 3
Bins: 60
Histogram
X axis: Wavelength / Angstrom
Y axis: 
Distribution: False
Instrument: None
Run start: not available
Run end:  not available
, Function=<mantid.fitfunctions.CompositeFunctionWrapper object at 0x7f10de8d6cc0>, CostFunction='Least squares')

sc.plot.plot(ds["sample"][Dim.Spectrum, 0])

sc.plot.plot(sc.sum(ds, Dim.Spectrum))

sc.plot.plot(stitched["sample"][Dim.Spectrum, 0])

sc.plot.plot(stitched["normalized_summed"])

sc.plot.plot(stitched["normalized_grpd"][Dim.Tof,34])

sc.plot.plot(stitched["normalized_wl"][Dim.Row, 5])

sc.plot.plot(sc.sum(stitched, Dim.Spectrum))